In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 5000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [3]:
lines = pd.read_csv('turkish.tsv', sep = '\t', header = None)
lines.rename(columns={0:'English',
                          1:'Turkish',
                          2:'Drop'}, 
                 inplace=True)
lines = lines.drop('Drop', axis =1)
lines = lines.values.tolist()

In [4]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in lines:
    count += 1

    if count > NUM_SENTENCES:
        break

    input_sentence, output = line[0], line[1]

    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))
print('\n')
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

num samples input: 5000
num samples output: 5000
num samples output input: 5000


Follow us.
Bizi izle. <eos>
<sos> Bizi izle.


In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

def clean_text(data):
    corpus = []
    for i in range(0,len(data)):
        review = re.sub('[^a-zA-Z]', ' ', data)
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]

        review = ' '.join(review)

        corpus.append(review)
    return corpus    

In [23]:
def tagg(decoder_input_sentence):
    bos = "<BOS> "
    eos = " <EOS>"
    tagged_sentence = [bos + text + eos for text in decoder_input_sentence] 
    return tagged_sentence 

In [ ]:
def create_vocabulary(encoder_input,decoder_input)
    vocab = set()

    for i,j in zip(encoder_input,decoder_input):
        vocab = vocab.union(set(i))
        vocab = vocab.union(set(j))

    return vocab+1    

In [ ]:
def assemble_data(input_data, output_data):
    all_data = []
    for i,j in zip(input_data, output_data):
         all_data.append(i,j)
    return all_data       

In [ ]:
def max_len(data):
    max_len_list = []
    for sent in data:
        max_len_list.append(len(sent))
    return max(max_len_list) + 1

In [1]:
tokenizer = Tokenizer(filters=[])

def vectorize(all_data, word_index = tokenizer.word_index, max_len = max_len): #Data has to be assembled ???
    iv = [] #Independent Variable
    dv = [] #Dependent Variable
    
    for question, answer in all_data:
        
        q = [word_index[word.lower()] for word in question] #q for questions
        a = [word_index[word.lower()] for word in answer]   #a for answers
        
        iv.append(q)
        dv.append(a)
        
    return pad_sequences(iv,max_len(iv)), pad_sequences(dv, max_len(dv))    
        

Using TensorFlow backend.


In [ ]:
def glove_100d_dictionary(GLOVE_DIR):
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector
  return embedding_matrix

def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  
  embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                              output_dim = EMBEDDING_DIM,
                              input_length = MAX_LEN,
                              weights = [embedding_matrix],
                              trainable = False)
  return embedding_layer

embedding_layer = embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix)


In [20]:
requirements = '''
   vocab_size, max_sequence_length, fit_on_text ??? 

   vectorization?
'''

In [21]:
requirements

'\n   vocab_size, max_sequence_length, fit_on_text ??? \n\n   vectorization?\n'

In [ ]:
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
encoder_embedding = embed_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

In [ ]:
decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
decoder_embedding = embed_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

In [ ]:
# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)